# VAZHI v0.4 - High-Tamil LoRA Training

**Goal**: Train improved Tamil LLM with 86.7% Tamil content (vs 2.6% in v0.2)

## Key Improvements over v0.2:
- ✅ 86.7% Tamil content (proper Tamil translations)
- ✅ Tamil-first bilingual format: "தமிழ் சொல் (English term)"
- ✅ 3,033 unique samples (0 duplicates)
- ✅ Fixed batch-compatible formatting function
- ✅ Standardized pack names (vazhi_*)
- ✅ 512 max tokens for inference

## Training Data:
| Split | Samples |
|-------|--------:|
| Train | 2,732 |
| Val | 301 |
| **Total** | **3,033** |

## Pack Distribution:
| Pack | Samples |
|------|--------:|
| vazhi_panpaadu (Culture) | 625 |
| vazhi_sattam (Legal) | 590 |
| vazhi_kaval (Security) | 467 |
| vazhi_maruthuvam (Healthcare) | 462 |
| vazhi_arasu (Government) | 452 |
| vazhi_kalvi (Education) | 437 |

---
**Runtime**: Select GPU (T4 or better) before running

**Data Files**: `data/v04/training/vazhi_v04_train.json` and `vazhi_v04_val.json`

## Cell 1: Install Dependencies

In [ ]:
# Install Unsloth and dependencies
!pip install unsloth
!pip install --no-deps trl peft accelerate bitsandbytes
!pip install datasets

## Cell 2: Upload Training Data

Upload these files from `data/v04/training/`:
- `vazhi_v04_train.json` (~3.4 MB)
- `vazhi_v04_val.json` (~385 KB)

In [ ]:
from google.colab import files

print("Upload vazhi_v04_train.json and vazhi_v04_val.json from data/v04/training/")
uploaded = files.upload()

# Verify uploaded files
import os
print("\nUploaded files:")
for f in os.listdir('.'):
    if f.endswith('.json'):
        print(f"  - {f} ({os.path.getsize(f) / 1024:.1f} KB)")

## Cell 3: Load Base Model with Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

# Model configuration
MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"
MAX_SEQ_LENGTH = 2048

# Load model in 4-bit for memory efficiency
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,  # Auto-detect
    load_in_4bit=True,
)

print("Model loaded successfully!")
print(f"Model parameters: {model.num_parameters():,}")

## Cell 4: Add LoRA Adapters

In [ ]:
# Configure LoRA for Tamil fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

print("LoRA adapters added!")
print(f"Trainable parameters: {model.num_parameters(only_trainable=True):,}")

## Cell 5: Define Tamil-First Chat Template

v0.4 uses Tamil-first format: "தமிழ்ச்சொல் (English term)" for technical vocabulary.

In [ ]:
# VAZHI v0.4 Tamil-first system prompt
SYSTEM_PROMPT = """நீங்கள் வழி (VAZHI), தமிழ்நாடு மக்களுக்கான AI உதவியாளர். தமிழில் தெளிவாகவும் உதவியாகவும் பதிலளியுங்கள். தொழில்நுட்ப சொற்களை தமிழில் முதலில் கூறி, பிறகு ஆங்கிலத்தில் அடைப்புக்குறிக்குள் குறிப்பிடுங்கள்."""

# Fixed formatting function - handles both single and batched examples
def format_prompt(examples):
    """Format training examples into Qwen chat template. Returns list."""
    texts = []
    
    # Handle both single example and batch
    if isinstance(examples['instruction'], list):
        # Batched
        for instruction, output in zip(examples['instruction'], examples['output']):
            text = f"""<|im_start|>system
{SYSTEM_PROMPT}<|im_end|>
<|im_start|>user
{instruction}<|im_end|>
<|im_start|>assistant
{output}<|im_end|>"""
            texts.append(text)
    else:
        # Single example
        text = f"""<|im_start|>system
{SYSTEM_PROMPT}<|im_end|>
<|im_start|>user
{examples['instruction']}<|im_end|>
<|im_start|>assistant
{examples['output']}<|im_end|>"""
        texts.append(text)
    
    return texts

# Test the template
test_example = {
    "instruction": "திருக்குறளின் முதல் குறள் என்ன?",
    "output": "முதல் குறள்: அகர முதல எழுத்தெல்லாம் ஆதி பகவன் முதற்றே உலகு."
}
print("Template example:")
print(format_prompt(test_example)[0])

## Cell 6: Load and Verify Training Data

In [ ]:
from datasets import Dataset
import json
import re

# Load v0.4 training data
with open("vazhi_v04_train.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)

with open("vazhi_v04_val.json", "r", encoding="utf-8") as f:
    val_data = json.load(f)

print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")

# Convert to HuggingFace Dataset
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

# Pack distribution
from collections import Counter
pack_counts = Counter(item.get('pack', 'unknown') for item in train_data + val_data)
print("\nPack Distribution:")
for pack, count in sorted(pack_counts.items(), key=lambda x: -x[1]):
    print(f"  {pack}: {count}")

In [ ]:
# Verify Tamil content in data
def tamil_pct(text):
    """Calculate percentage of Tamil characters."""
    if not text:
        return 0
    tamil = len(re.findall(r'[\u0B80-\u0BFF]', text))
    total = len(re.findall(r'[\u0B80-\u0BFF\w]', text))
    return (tamil / total * 100) if total > 0 else 0

# Check Tamil percentage
pcts = [tamil_pct(s['output']) for s in train_data]
print(f"Tamil Content Verification:")
print(f"  Average: {sum(pcts)/len(pcts):.1f}%")
print(f"  Min: {min(pcts):.1f}%")
print(f"  Max: {max(pcts):.1f}%")
print(f"  Samples < 50% Tamil: {sum(1 for p in pcts if p < 50)}")

# Show sample entry
print("\nSample entry:")
print(f"  Pack: {train_dataset[0]['pack']}")
print(f"  Instruction: {train_dataset[0]['instruction'][:80]}...")
print(f"  Output: {train_dataset[0]['output'][:100]}...")

## Cell 7: Configure Training Arguments

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

# Training configuration - optimized for T4 GPU
training_args = TrainingArguments(
    output_dir="./vazhi-v04-lora",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,  # Effective batch size = 8
    learning_rate=2e-4,
    warmup_steps=50,
    optim="adamw_8bit",
    weight_decay=0.01,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=10,
    save_steps=200,
    save_total_limit=3,
    eval_strategy="steps",
    eval_steps=200,
    seed=42,
    report_to="none",
)

print("Training arguments configured!")
print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")

## Cell 8: Initialize Trainer

In [ ]:
# Create the trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    formatting_func=format_prompt,
    max_seq_length=MAX_SEQ_LENGTH,
    args=training_args,
)

steps_per_epoch = len(train_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)
total_steps = steps_per_epoch * training_args.num_train_epochs

print("Trainer initialized!")
print(f"Training samples: {len(train_dataset)}")
print(f"Steps per epoch: {steps_per_epoch}")
print(f"Total steps: {total_steps}")

## Cell 9: Train the Model

**Estimated time**: ~45-60 minutes on T4 GPU

In [ ]:
import time

print("="*60)
print("VAZHI v0.4 Training - High Tamil Content")
print("="*60)
print(f"Training {len(train_dataset)} samples with 86.7% Tamil content")
print(f"Expected: ~{steps_per_epoch * 3} steps over 3 epochs")
print("="*60)

start_time = time.time()

# Train!
trainer.train()

end_time = time.time()
training_time = end_time - start_time

print("\n" + "="*60)
print("Training Complete!")
print("="*60)
print(f"Total training time: {training_time:.2f} seconds ({training_time/60:.1f} minutes)")

## Cell 10: Save and Download the Model

In [ ]:
# Save the LoRA adapter
model.save_pretrained("vazhi-v04-lora-final")
tokenizer.save_pretrained("vazhi-v04-lora-final")

print("Model saved to vazhi-v04-lora-final/")

# Download the model files
import shutil

shutil.make_archive("vazhi-v04-lora-final", 'zip', "vazhi-v04-lora-final")
files.download("vazhi-v04-lora-final.zip")
print("Model downloaded!")

## Cell 11: Define Inference Function

In [ ]:
# Set model to inference mode
FastLanguageModel.for_inference(model)

def vazhi_chat(user_message: str, max_tokens: int = 512) -> str:
    """
    Chat with VAZHI v0.4 model.

    Args:
        user_message: User's question in Tamil/Tanglish/English
        max_tokens: Maximum response length (default 512)

    Returns:
        Model's response
    """
    prompt = f"""<|im_start|>system
{SYSTEM_PROMPT}<|im_end|>
<|im_start|>user
{user_message}<|im_end|>
<|im_start|>assistant
"""

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # Extract assistant response (improved parsing)
    if "<|im_start|>assistant" in response:
        response = response.split("<|im_start|>assistant")[-1]

    # Clean up markers
    response = response.replace("<|im_end|>", "").strip()
    response = response.replace("<|im_start|>", "").strip()

    return response

print("Inference function ready!")

## Cell 12: Comprehensive Test Across All Packs

In [ ]:
# Test questions across all packs
test_questions = [
    # Culture - Thirukkural
    ("Culture", "திருக்குறளின் முதல் குறள் என்ன?"),
    ("Culture", "அகர முதல எழுத்தெல்லாம் என்ற குறளின் பொருள் என்ன?"),

    # Culture - Siddhars
    ("சித்தர்கள்", "சித்தர்கள் யார்? முக்கிய சித்தர்களின் பெயர்கள் சொல்லுங்கள்."),
    ("Culture", "அகத்தியர் பற்றி சொல்லுங்கள்"),

    # Security
    ("Security", "மோசடி செய்தியை எப்படி கண்டறிவது?"),
    ("Security", "ஒட்டுமுறை கடவுச்சொல்லை (OTP) யாருக்கும் பகிர்ந்துக்கொள்ளக்கூடாது என்?"),

    # Government
    ("Government", "முதலமைச்சர் காப்பீட்டு அட்டை எப்படி பெறுவது?"),

    # Education
    ("Education", "நீட் தேர்வுக்கு எப்படி தயாராவது?"),

    # Legal
    ("Legal", "தகவல் அறியும் உரிமைச் சட்டம் என்றால் என்ன?"),

    # Healthcare
    ("Healthcare", "அரசு மருத்துவமனையில் இலவச சிகிச்சை கிடைக்குமா?"),
]

print("="*60)
print("VAZHI v0.4 - Comprehensive High-Tamil Test")
print("="*60)

for category, q in test_questions:
    print(f"\n[{category}] Q: {q}")
    response = vazhi_chat(q)
    pct = tamil_pct(response)
    print(f"A: {response[:300]}..." if len(response) > 300 else f"A: {response}")
    print(f"Tamil %: {pct:.1f}%")
    print("-"*50)

## Cell 13: Thirukkural Validation

Critical test - v0.1 failed on Thirukkural accuracy.

In [ ]:
# Thirukkural accuracy test
thirukkural_tests = [
    {
        "question": "திருக்குறளின் முதல் குறள் என்ன?",
        "expected_contains": ["அகர முதல", "எழுத்தெல்லாம்", "ஆதி", "பகவன்"]
    },
    {
        "question": "கற்றதனால் ஆய பயனென்கொல் என்ற குறள் முழுமையாக சொல்லுங்கள்",
        "expected_contains": ["வாலறிவன்", "நற்றாள்", "தொழாஅர்"]
    },
    {
        "question": "பிறவிப் பெருங்கடல் என்ற குறள் என்ன?",
        "expected_contains": ["பிறவி", "கடல்", "நீந்து", "இறைவன்"]
    },
]

print("="*60)
print("திருக்குறள் Accuracy Validation")
print("="*60)

passed = 0
for test in thirukkural_tests:
    response = vazhi_chat(test["question"])
    matches = sum(1 for keyword in test["expected_contains"] if keyword in response)
    total = len(test["expected_contains"])
    status = "✅ PASS" if matches >= total * 0.75 else "❌ FAIL"

    if matches >= total * 0.75:
        passed += 1

    print(f"\nQ: {test['question']}")
    print(f"A: {response[:300]}..." if len(response) > 300 else f"A: {response}")
    print(f"Keywords found: {matches}/{total} {status}")

print(f"\n{'='*60}")
print(f"திருக்குறள் Tests: {passed}/{len(thirukkural_tests)} passed")
print("="*60)

## Cell 14: v0.2 vs v0.4 Comparison Summary

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════╗
║              VAZHI v0.4 Training Complete!                   ║
╠══════════════════════════════════════════════════════════════╣
║  Key improvements:                                           ║
║  ┌─────────────────────┬───────────┬───────────┐             ║
║  │ Feature             │ v0.2      │ v0.4      │             ║
║  ├─────────────────────┼───────────┼───────────┤             ║
║  │ Tamil content       │ ~2.6%     │ 86.7%     │             ║
║  │ Training samples    │ 2,860*    │ 3,033     │             ║
║  │ Bilingual format    │ Mixed     │ Tamil 1st │             ║
║  │ Tanglish            │ Heavy     │ Minimal   │             ║
║  │ Data format         │ Mixed     │ Standard  │             ║
║  │ Duplicates          │ 320       │ 0         │             ║
║  └─────────────────────┴───────────┴───────────┘             ║
║  * v0.2 reported 3,180 but had 320 duplicate IDs             ║
║                                                              ║
║  Pack Distribution (all standardized as vazhi_*):            ║
║  - vazhi_panpaadu (Culture): 625 samples                     ║
║  - vazhi_sattam (Legal): 590 samples                         ║
║  - vazhi_kaval (Security): 467 samples                       ║
║  - vazhi_maruthuvam (Healthcare): 462 samples                ║
║  - vazhi_arasu (Government): 452 samples                     ║
║  - vazhi_kalvi (Education): 437 samples                      ║
║                                                              ║
║  Files saved: vazhi-v04-lora-final.zip                       ║
║                                                              ║
║  Next steps:                                                 ║
║  1. Upload to HuggingFace Hub                                ║
║  2. Deploy to HuggingFace Spaces                             ║
║  3. Test Flutter app with new model                          ║
╚══════════════════════════════════════════════════════════════╝
""")

## Cell 15: (Optional) Upload to HuggingFace Hub

Run this cell to push your model directly to HuggingFace Hub.

In [ ]:
# Optional: Push to HuggingFace Hub
# Uncomment and run if you want to upload directly

# from huggingface_hub import login, HfApi
#
# # Login to HuggingFace
# login()  # This will prompt for your token
#
# # Push model to Hub
# model.push_to_hub("cryptoyogillc/vazhi-tamil-v04")
# tokenizer.push_to_hub("cryptoyogillc/vazhi-tamil-v04")
#
# print("Model uploaded to HuggingFace Hub!")